# Question: Do Players Who Radically Change Their Launch Angle Change Their Exit Velocity

Recreating an analysis done by Tom Tango (see http://tangotiger.com/index.php/site/comments/statcast-lab-do-players-who-radically-change-their-launch-angle-change-thei#comments)

In [1]:
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# import the data
bip_2018 = pd.read_csv("/Users/chrisjackson/sports/baseball/data/pitch_data_2018.csv")
bip_2019 = pd.read_csv("/Users/chrisjackson/sports/baseball/data/pitch_data_2019.csv")

# keep only balls put into play
bip_list = ['hit_into_play', 'hit_into_play_score', 'hit_into_play_no_out']
bip_2018 = bip_2018[bip_2018['description'].isin(bip_list)]
bip_2019 = bip_2019[bip_2019['description'].isin(bip_list)]

# get a list of batters with AT LEAST 100 balls put into play for each season
bip_2018['batter'] = bip_2018['batter'].astype(int)
bip_2018_dict = dict(Counter(bip_2018['batter']))
bip_2018_100_list = [k for k, v in bip_2018_dict.items() if v >= 100]

bip_2019['batter'] = bip_2019['batter'].astype(int)
bip_2019_dict = dict(Counter(bip_2019['batter']))
bip_2019_100_list = [k for k, v in bip_2019_dict.items() if v >= 100]

# find the list of batters that had AT LEAST 100 balls put into play in 2018 AND 2019
bip_2018_2019_100_list = [x for x in bip_2018_100_list if x in bip_2019_100_list]

# keep only these batters in the data
bip_2018 = bip_2018[bip_2018['batter'].isin(bip_2018_2019_100_list)]
bip_2019 = bip_2019[bip_2019['batter'].isin(bip_2018_2019_100_list)]

bip_2018 = bip_2018[['batter', 'launch_speed', 'launch_angle']]
#bip_2018.columns = ['batter', 'launch_speed_2018', 'launch_angle_2018']
print(bip_2018.shape)

bip_2019 = bip_2019[['batter', 'launch_speed', 'launch_angle']]
#bip_2019.columns = ['batter', 'launch_speed_2019', 'launch_angle_2019']
print(bip_2019.shape)

bip_2018.head()

(88433, 3)
(68722, 3)


,batter,launch_speed,launch_angle
11,596115,104.7,24.0
20,571448,107.1,23.0
21,571771,81.0,-11.0
22,624577,65.7,-16.0
24,641355,96.3,-8.0


In [3]:
def count_launch_angles(df):
    
    la = df[['batter', 'launch_angle']]

    la['la_8_to_32'] = (la['launch_angle'] >= 8.0) & (la['launch_angle'] <= 32.0)
    la['la_below_8'] = la['launch_angle'] < 8.0
    la['la_above_32'] = la['launch_angle'] > 32.0

    la.drop('launch_angle', axis=1, inplace=True)

    la = pd.DataFrame(la.groupby('batter').sum())
    
    return la

la_2018 = count_launch_angles(bip_2018)
la_2018.columns = ['la_8_to_32_2018', 'la_below_8_2018', 'la_above_32_2018']
la_2018.reset_index(inplace=True, drop=False)
la_2018['total'] = (la_2018['la_8_to_32_2018'] + la_2018['la_below_8_2018'] + la_2018['la_above_32_2018'])
la_2018['la_8_to_32_2018'] = la_2018['la_8_to_32_2018'] / la_2018['total']
la_2018['la_below_8_2018'] = la_2018['la_below_8_2018'] / la_2018['total']
la_2018['la_above_32_2018'] = la_2018['la_above_32_2018'] / la_2018['total']
la_2018.drop('total', axis=1, inplace=True)

la_2019 = count_launch_angles(bip_2019)
la_2019.columns = ['la_8_to_32_2019', 'la_below_8_2019', 'la_above_32_2019']
la_2019.reset_index(inplace=True, drop=False)
la_2019['total'] = (la_2019['la_8_to_32_2019'] + la_2019['la_below_8_2019'] + la_2019['la_above_32_2019'])
la_2019['la_8_to_32_2019'] = la_2019['la_8_to_32_2019'] / la_2019['total']
la_2019['la_below_8_2019'] = la_2019['la_below_8_2019'] / la_2019['total']
la_2019['la_above_32_2019'] = la_2019['la_above_32_2019'] / la_2019['total']
la_2019.drop('total', axis=1, inplace=True)

la_2018_2019 = pd.merge(la_2018, la_2019, on='batter')

la_2018_2019.head()

,batter,la_8_to_32_2018,la_below_8_2018,la_above_32_2018,la_8_to_32_2019,la_below_8_2019,la_above_32_2019
0,405395,0.362283,0.414392,0.223325,0.260450,0.472669,0.266881
1,408234,0.305556,0.555556,0.138889,0.381410,0.435897,0.182692
2,425772,0.393939,0.363636,0.242424,0.304000,0.400000,0.296000
3,425783,0.366093,0.501229,0.132678,0.332237,0.493421,0.174342
4,425877,0.390977,0.380952,0.228070,0.419087,0.381743,0.199170


In [4]:
la_2018_2019['la_8_to_32_diff'] = la_2018_2019['la_8_to_32_2019'] - la_2018_2019['la_8_to_32_2018']
la_2018_2019['la_below_8_diff'] = la_2018_2019['la_below_8_2019'] - la_2018_2019['la_below_8_2018']
la_2018_2019['la_above_32_diff'] = la_2018_2019['la_above_32_2019'] - la_2018_2019['la_above_32_2018']

la_2018_2019 = la_2018_2019[['batter', 'la_below_8_diff', 'la_8_to_32_diff', 'la_above_32_diff']]

la_2018_2019.head(10)

,batter,la_below_8_diff,la_8_to_32_diff,la_above_32_diff
0,405395,0.058277,-0.101833,0.043556
1,408234,-0.119658,0.075855,0.043803
2,425772,0.036364,-0.089939,0.053576
3,425783,-0.007807,-0.033857,0.041664
4,425877,0.000790,0.028110,-0.028900
5,429664,0.005703,-0.060279,0.054576
6,429665,-0.050364,0.001595,0.048770
7,430945,0.021185,-0.044513,0.023328
8,431145,0.016378,0.017085,-0.033463
9,434158,0.019613,-0.044609,0.024996


In [6]:
def categorize_batter(row):
    below_8 = row['la_below_8_diff']
    between_8_32 = row['la_8_to_32_diff']
    above_32 = row['la_above_32_diff']
    
    if below_8 > 0 and between_8_32 < 0 and above_32 < 0:
        return 1
    elif below_8 < 0 and between_8_32 > 0 and above_32 < 0:
        return 2
    elif below_8 < 0 and between_8_32 < 0 and above_32 > 0:
        return 3
    elif below_8 > 0 and between_8_32 > 0 and above_32 < 0:
        return 4
    elif below_8 < 0 and between_8_32 > 0 and above_32 > 0:
        return 5
    else:
        return 6
    
la_2018_2019['la_category'] = la_2018_2019.apply(categorize_batter, axis=1)

la_2018_2019.head()

,batter,la_below_8_diff,la_8_to_32_diff,la_above_32_diff,la_category
0,405395,0.058277,-0.101833,0.043556,6
1,408234,-0.119658,0.075855,0.043803,5
2,425772,0.036364,-0.089939,0.053576,6
3,425783,-0.007807,-0.033857,0.041664,3
4,425877,0.000790,0.028110,-0.028900,4


In [7]:
Counter(la_2018_2019['la_category'])

Counter({6: 48, 5: 70, 3: 40, 4: 32, 1: 41, 2: 41})

In [16]:
cat_1_list = la_2018_2019[la_2018_2019['la_category'] == 1]['batter'].tolist()
cat_2_list = la_2018_2019[la_2018_2019['la_category'] == 2]['batter'].tolist()
cat_3_list = la_2018_2019[la_2018_2019['la_category'] == 3]['batter'].tolist()
cat_4_list = la_2018_2019[la_2018_2019['la_category'] == 4]['batter'].tolist()
cat_5_list = la_2018_2019[la_2018_2019['la_category'] == 5]['batter'].tolist()
cat_6_list = la_2018_2019[la_2018_2019['la_category'] == 6]['batter'].tolist()

In [14]:
bip_2018_avg = pd.DataFrame(bip_2018.groupby('batter').mean())
bip_2018_avg.reset_index(inplace=True, drop=False)

bip_2019_avg = pd.DataFrame(bip_2019.groupby('batter').mean())
bip_2019_avg.reset_index(inplace=True, drop=False)

bip_2018_2019_avg = pd.merge(bip_2018_avg, bip_2019_avg, on='batter')

bip_2018_2019_avg['launch_angle_diff'] = bip_2018_2019_avg['launch_angle_y'] - bip_2018_2019_avg['launch_angle_x']
bip_2018_2019_avg['launch_speed_diff'] = bip_2018_2019_avg['launch_speed_y'] - bip_2018_2019_avg['launch_speed_x']

bip_2018_2019_avg = bip_2018_2019_avg[['batter', 'launch_angle_diff', 'launch_speed_diff']]

bip_2018_2019_avg.head()

,batter,launch_angle_diff,launch_speed_diff
0,405395,-1.833492,-0.978679
1,408234,5.335114,-3.899074
2,425772,2.406000,-0.254958
3,425783,1.665427,2.145857
4,425877,-1.839557,-1.464653


In [17]:
# average change in launch angle for category 1 hitters
print(bip_2018_2019_avg[bip_2018_2019_avg['batter'].isin(cat_1_list)]['launch_angle_diff'].mean())

# average change in launch angle for category 2 hitters
print(bip_2018_2019_avg[bip_2018_2019_avg['batter'].isin(cat_2_list)]['launch_angle_diff'].mean())

# average change in launch angle for category 3 hitters
print(bip_2018_2019_avg[bip_2018_2019_avg['batter'].isin(cat_3_list)]['launch_angle_diff'].mean())

# average change in launch angle for category 4 hitters
print(bip_2018_2019_avg[bip_2018_2019_avg['batter'].isin(cat_4_list)]['launch_angle_diff'].mean())

# average change in launch angle for category 5 hitters
print(bip_2018_2019_avg[bip_2018_2019_avg['batter'].isin(cat_5_list)]['launch_angle_diff'].mean())

# average change in launch angle for category 6 hitters
print(bip_2018_2019_avg[bip_2018_2019_avg['batter'].isin(cat_6_list)]['launch_angle_diff'].mean())

-3.3754060693679437
0.46621960792555184
2.5845739887676977
-1.830584144235515
3.4424432856481215
-0.4828576053958824


In [18]:
# average change in exit velocity for category 1 hitters
print(bip_2018_2019_avg[bip_2018_2019_avg['batter'].isin(cat_1_list)]['launch_speed_diff'].mean())

# average change in exit velocity for category 2 hitters
print(bip_2018_2019_avg[bip_2018_2019_avg['batter'].isin(cat_2_list)]['launch_speed_diff'].mean())

# average change in exit velocity for category 3 hitters
print(bip_2018_2019_avg[bip_2018_2019_avg['batter'].isin(cat_3_list)]['launch_speed_diff'].mean())

# average change in exit velocity for category 4 hitters
print(bip_2018_2019_avg[bip_2018_2019_avg['batter'].isin(cat_4_list)]['launch_speed_diff'].mean())

# average change in exit velocity for category 5 hitters
print(bip_2018_2019_avg[bip_2018_2019_avg['batter'].isin(cat_5_list)]['launch_speed_diff'].mean())

# average change in exit velocity for category 6 hitters
print(bip_2018_2019_avg[bip_2018_2019_avg['batter'].isin(cat_6_list)]['launch_speed_diff'].mean())

0.14379663896284903
0.9845186808402008
0.5122645041404457
0.6105574919973877
0.6317432131123358
-0.4933448924462889
